In [1]:
import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from data_manager import *
from metadata import *

import matplotlib.pyplot as plt
%matplotlib inline

import time

Setting environment for Gordon


In [2]:
from skimage.transform import resize

In [3]:
resection_dir = '/home/yuncong/csd395/CSHL_volumes_resection'

In [4]:
# for stack in all_stacks:
for stack in ['MD589', 'MD585', 'MD594']:

    first_sec, last_sec = section_range_lookup[stack]
    
#     first_sec = 200
#     last_sec = 220

    for downsample in [4]:
    
        xy_pixel_distance = xy_pixel_distance_lossless * downsample
        voxel_z_size = section_thickness / xy_pixel_distance
        print 'voxel size:', xy_pixel_distance, xy_pixel_distance, voxel_z_size, 'um'

        full_width, full_height = DataManager.get_image_dimension(stack)
        xdim = full_width/downsample
        ydim = full_height/downsample
        z_end = int(np.ceil((last_sec+1)*voxel_z_size))
        z_begin = int(np.floor(first_sec*voxel_z_size))
        zdim = z_end + 1 - z_begin

        print xdim, ydim, zdim

        volume = np.zeros((ydim, xdim, zdim), np.uint8)
    #     volume = np.zeros((1000, 1000, zdim), np.uint8)

        bar = show_progress_bar(first_sec, last_sec)

        for sec in range(first_sec, last_sec+1):
            bar.value = sec
    #         t = time.time()
            if downsample == 32:
                img = img_as_ubyte(rgb2gray(imread(os.environ['DATA_DIR'] + '/%(stack)s_thumbnail_aligned_masked_cropped/%(stack)s_%(slice)04d_thumbnail_aligned_masked_cropped.png' \
                                % {'stack': stack, 'slice': sec})))
            else:
                img = img_as_ubyte(rgb2gray(imread(DataManager.get_image_filepath(stack=stack, section=sec))[::downsample, ::downsample]))
                mask_tb = DataManager.load_thumbnail_mask(stack, sec)
                mask = resize(mask_tb, img.shape[:2]) > .5
                img[~mask] = 0

    #         sys.stderr.write('%.2f seconds.\n' % (time.time() - t))
            z1 = sec * voxel_z_size
            z2 = (sec + 1) * voxel_z_size
    #         volume[:, :, int(z1)-z_begin:int(z2)+1-z_begin] = img[10000:11000, 10000:11000, None]
            volume[:, :, int(z1)-z_begin:int(z2)+1-z_begin] = img[..., None]

        bp.pack_ndarray_file(volume, volume_dir + '/%(stack)s/%(stack)s_down%(downsample)dVolume.bp' % {'stack': stack, 
                                                                                                        'downsample': downsample})
        del volume

#     for x in range(volume.shape[1]):
#         create_if_not_exists(os.path.join(resection_dir, 'highres_coronal'))
#         imsave(os.path.join(resection_dir, 'highres_coronal', '%s_highresVolume_coronal_%03d.jpg'%(stack, x)), volume[:,x,:])

#     for y in range(volume.shape[0]):
#         create_if_not_exists(os.path.join(resection_dir, 'highres_horizontal'))
#         imsave(os.path.join(resection_dir, 'highres_horizontal', '%s_highresVolume_horizontal_%03d.jpg'%(stack, y)), volume[y,:,:])

#     for z in range(volume.shape[2]):
#         create_if_not_exists(os.path.join(resection_dir, 'highres_sagittal'))
#         imsave(os.path.join(resection_dir, 'highres_sagittal', '%s_highresVolume_sagittal_%03d.jpg'%(stack, z)), volume[:,:,z])

#     del volume

voxel size: 3.68 3.68 5.4347826087 um
1996

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/PIL/Image.py:2224: DecompressionBombWarning: Image size (187528192 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


 1468 1502
voxel size: 3.68 3.68 5.4347826087 um
2108 1632 1470
voxel size: 3.68 3.68 5.4347826087 um
2056

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/PIL/Image.py:2224: DecompressionBombWarning: Image size (220176384 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/PIL/Image.py:2224: DecompressionBombWarning: Image size (190533632 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


 1448 1480
